While converting to CoreML there is an option to set image preprocessing parameters. Channel wise bias and an overall scale is supported, which is quite common. However, some models may require a per channel scale parameter. 
This can be implemented by adding a "scale" layer in the beginning of the network, after conversion. Let us see how this can be done by directly editing the mlmodel spec which is in protobuf format.

In [19]:
import coremltools
from keras.layers import *
from keras.models import Sequential
import numpy as np
from PIL import Image
import copy

In [20]:
# Define a toy Keras network and convert to CoreML
input_shape = (50, 50, 3)
model = Sequential()
model.add(Cropping2D(cropping=((5,5),(5,5)), input_shape=input_shape))


mlmodel = coremltools.converters.keras.convert(model,
                                              image_input_names='input1',
                                              red_bias=-10.0, 
                                              green_bias=-10.0, 
                                              blue_bias=-10.0,
                                              image_scale=5.0)

0 : cropping2d_3_input, <keras.engine.topology.InputLayer object at 0x12b310090>
1 : cropping2d_3, <keras.layers.convolutional.Cropping2D object at 0x12825b390>


In [21]:
spec = mlmodel.get_spec()
print(spec.description)

input {
  name: "input1"
  type {
    imageType {
      width: 50
      height: 50
      colorSpace: RGB
    }
  }
}
output {
  name: "output1"
  type {
    multiArrayType {
      shape: 3
      shape: 40
      shape: 40
      dataType: DOUBLE
    }
  }
}



In [22]:
# Lets call predict with an all constant image input
x = 100.0 * np.ones((3,50,50))
x = x.astype(np.uint8)
x_transpose = np.transpose(x, [1,2,0]) # PIL Image requires the format to be [H,W,C]
im = Image.fromarray(x_transpose)

y = mlmodel.predict({'input1': im}, useCPUOnly=True)['output1']
print('output along channel at [0,0]: ', y[:,0,0])

('output along channel at [0,0]: ', array([490., 490., 490.]))


As expected the output values are 490. That is, ${\textrm {scale}} * {\textrm {input}} + {\textrm {bias}}, \;\;{\textrm{i.e.,}}\;\;5*100 -10 = 490$.
Let us insert a channel dependent scale layer in the beginning of the network, before the crop layer.

In [23]:
# get NN portion of the spec
nn_spec = spec.neuralNetwork
layers = nn_spec.layers # this is a list of all the layers
layers_copy = copy.deepcopy(layers) # make a copy of the layers, these will be added back later
del nn_spec.layers[:] # delete all the layers

In [24]:
# add a scale layer now
# since mlmodel is in protobuf format, we can add proto messages directly
# To look at more examples on how to add other layers: see "builder.py" file in coremltools repo
scale_layer = nn_spec.layers.add()
scale_layer.name = 'scale_layer'
scale_layer.input.append('input1')
scale_layer.output.append('input1_scaled')
params = scale_layer.scale
params.scale.floatValue.extend([1.0, 2.0, 3.0]) # scale values for RGB
params.shapeScale.extend([3,1,1]) # shape of the scale vector 

# now add back the rest of the layers (which happens to be just one in this case: the crop layer)
nn_spec.layers.extend(layers_copy)

# need to also change the input of the crop layer to match the output of the scale layer
nn_spec.layers[1].input[0] = 'input1_scaled'

In [25]:
# Lets run the model again
mlmodel = coremltools.models.MLModel(spec)
y = mlmodel.predict({'input1': im}, useCPUOnly=True)['output1']
print('output along channel at [0,0]: ', y[:,0,0])

('output along channel at [0,0]: ', array([ 490.,  980., 1470.]))


As expected the values are scaled by 1.0, 2.0, 3.0: the parameters of the scale layer. 